In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")

data.shape

(20758, 18)

In [3]:
data.head(8)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
5,5,Male,18.128249,1.748524,51.552595,yes,yes,2.919751,3.000000,Sometimes,no,2.137550,no,1.930033,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
6,6,Male,29.883021,1.754711,112.725005,yes,yes,1.991240,3.000000,Sometimes,no,2.000000,no,0.000000,0.696948,Sometimes,Automobile,Obesity_Type_II
7,7,Male,29.891473,1.750150,118.206565,yes,yes,1.397468,3.000000,Sometimes,no,2.000000,no,0.598655,0.000000,Sometimes,Automobile,Obesity_Type_II


In [4]:
# There are no missing values

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [5]:
# There are seven categories target

data.loc[:, 'NObeyesdad'].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

In [6]:
data.loc[:, 'NObeyesdad'].value_counts() /  len(data.loc[:, 'NObeyesdad'])

NObeyesdad
Obesity_Type_III       0.194913
Obesity_Type_II        0.156470
Normal_Weight          0.148473
Obesity_Type_I         0.140187
Insufficient_Weight    0.121544
Overweight_Level_II    0.121495
Overweight_Level_I     0.116919
Name: count, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

import optuna
import optuna.visualization as vis

# Models to try
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

X = round(data.drop(['NObeyesdad', 'id'], axis = 1), 2)
y = data['NObeyesdad']

le = LabelEncoder()
y_le = le.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(X, y_le, train_size=0.8, test_size=0.2, random_state=0)

features_cat = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

features_num = X_train.select_dtypes(include=[np.number]).columns.tolist()

transformer_num = make_pipeline(
    StandardScaler()
)
transformer_cat = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'), 
    StandardScaler(with_mean=False)
)

preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)


In [8]:
models = {
    'XGBClassifier':XGBClassifier(),
    'LGBMClassifier':LGBMClassifier(verbose = -1),
    'CatBoostClassifier': CatBoostClassifier(verbose = False),
}

list_modelName=[]
list_predict=[]
for model_name, model in models.items():
    print("Model: " + model_name)
    pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                           ("classifier", model)])
    #pred=pipe.fit(X_train, y_train).predict(X_val) #predict
    
    #score=accuracy_score(y_val, pred)
    score = cross_val_score(pipe, X_train, y_train, cv=5) 
    
    list_modelName.append(model_name) #model name
    list_predict.append(score.mean()) #model score

models_scores=pd.DataFrame([list_modelName, list_predict]).T.rename(columns=({0:'Model', 1:'Score'})).sort_values('Score', ascending = False)
models_scores

Model: XGBClassifier
Model: LGBMClassifier
Model: CatBoostClassifier


,Model,Score
0,XGBClassifier,0.905095
2,CatBoostClassifier,0.904372
1,LGBMClassifier,0.904131


In [9]:
pd.DataFrame(preprocessor.fit_transform(X_train), columns = preprocessor.get_feature_names_out())

,pipeline-1__Age,pipeline-1__Height,pipeline-1__Weight,pipeline-1__FCVC,pipeline-1__NCP,pipeline-1__CH2O,pipeline-1__FAF,pipeline-1__TUE,pipeline-2__Gender_Female,pipeline-2__Gender_Male,...,pipeline-2__SCC_no,pipeline-2__SCC_yes,pipeline-2__CALC_Frequently,pipeline-2__CALC_Sometimes,pipeline-2__CALC_no,pipeline-2__MTRANS_Automobile,pipeline-2__MTRANS_Bike,pipeline-2__MTRANS_Motorbike,pipeline-2__MTRANS_Public_Transportation,pipeline-2__MTRANS_Walking
0,-0.489353,0.458435,1.712712,1.039251,0.338761,-0.327625,0.713888,-0.059315,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0
1,0.377707,-0.570225,-0.940764,1.039251,0.338761,-0.048390,-1.174537,2.302033,2.000001,0.000000,...,5.511307,0.0,0.0,0.000000,2.316796,2.65584,0.0,0.0,0.000000,0.0
2,-0.195671,-0.570225,0.417084,-0.840209,-0.113802,0.099441,-1.174537,1.237764,0.000000,2.000001,...,5.511307,0.0,0.0,0.000000,2.316796,0.00000,0.0,0.0,2.513785,0.0
3,-0.424673,0.572730,1.729026,1.039251,0.338761,-0.508306,0.582416,0.156865,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0
4,-1.020776,0.915617,0.766505,-0.840209,0.338761,-1.690950,0.020669,0.639112,0.000000,2.000001,...,5.511307,0.0,0.0,0.000000,2.316796,0.00000,0.0,0.0,2.513785,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16601,-1.195586,-1.141703,-1.433975,-2.719668,0.338761,-0.048390,-1.174537,2.302033,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0
16602,0.377707,-0.798816,0.872735,1.039251,0.338761,1.101402,-1.174537,-0.375270,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0
16603,-1.160624,2.630050,-1.446116,-2.569311,1.201460,-0.048390,1.215875,0.639112,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0
16604,0.377707,-0.798816,0.911433,1.039251,0.338761,0.855018,-1.174537,-0.657966,2.000001,0.000000,...,5.511307,0.0,0.0,2.242421,0.000000,0.00000,0.0,0.0,2.513785,0.0


In [10]:
le.inverse_transform([0, 1, 2, 3, 4, 5, 6])

array(['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II'], dtype=object)

In [11]:
import optuna
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score

def objective(trial):
    
    params = {
         "objective": "multiclass",
         "metric": "multi_logloss",
         "verbosity": -1,
         "random_state": 0,
         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
         "n_estimators": trial.suggest_int("n_estimators", 100, 800, step=100),
         "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
         "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
         "max_depth": trial.suggest_int("max_depth", 2, 16, step=1),
         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9, step=0.1),
         "subsample": trial.suggest_float("subsample", 0.7, 1, step=0.1),
         "min_child_samples": trial.suggest_int("min_child_samples", 5, 50, step=5),
         "num_leaves" : trial.suggest_int('num_leaves', 5, 100, step=5)
     }
          
    LGBM_class = LGBMClassifier(**params)
    
    pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                       ("classifier", LGBM_class)])
    
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    
    score=accuracy_score(y_val, y_pred)
    
    score = cross_val_score(pipe, X_train, y_train, cv=5) 
    
    return score.mean()

In [12]:
# LGBM optuna
#study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=30)
#print('Number of finished trials:', len(study.trials))
#print('Best trial:', study.best_trial.params)

In [13]:
#import optuna.visualization as vis

#vis.plot_param_importances(study)

In [14]:
LGBM_class = LGBMClassifier(n_estimators = 800, 
                            lambda_l1 = 0,
                            lambda_l2 = 100,
                            colsample_bytree = 0.4, 
                            learning_rate = 0.069, 
                            max_depth = 14, 
                            num_leaves = 85,
                            min_child_samples = 45,
                            subsample = 0.79,
                            class_weight = 'balanced',
                           verbose = -1)

pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                       ("classifier", LGBM_class)])
pipe.fit(X, y_le)

y_pred = pipe.predict(X_val)

#score = accuracy_score(y_val, y_pred)
score = cross_val_score(pipe, X, y, cv=5) 
print(f'Accuracy Score on Test set: {score.mean()}') 


Accuracy Score on Test set: 0.9093845344042734


In [15]:
XGB_class = XGBClassifier(max_depth = 8, 
                          subsample = 0.7,
                          n_estimators = 800, 
                          eta = 0.06999999999999999, 
                          reg_alpha = 5, 
                          reg_lambda = 60, 
                          min_child_weight = 4, 
                          colsample_bytree = 0.6)

pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                       ("classifier", XGB_class)])
pipe.fit(X, y_le)

y_pred = pipe.predict(X_val)

#score = accuracy_score(y_val, y_pred)
score = cross_val_score(pipe, X, y_le, cv=5) 
print(f'Accuracy Score on Test set: {score.mean()}') 


Accuracy Score on Test set: 0.908517354733567


In [16]:
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")

pipe = Pipeline(steps=[("preprocessor", preprocessor), 
                       ("classifier", XGB_class)])

pipe.fit(X, y_le)
p = pipe.predict(test)

d = {'id': test['id'], 'NObeyesdad': le.inverse_transform(p)} 

predictions = pd.DataFrame(d)
predictions.to_csv("/kaggle/working/submission.csv", index=False) 